In [3]:
import pickle
import json
import redshift_connector
import pandas as pd
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)
import numpy as np
import unidecode
import re

from collections import Counter
from math import ceil
from langdetect import detect
from random import sample

## Exploring the ROR Data to Create Artificial Training Data

In [4]:
# Data was downloaded from the ROR website for the date seen in the file string below
# -------> https://ror.readme.io/docs/data-dump
ror = pd.read_json("./v1.19-2023-02-16-ror-data.json")

In [63]:
ror['alias_len'] = ror['aliases'].apply(len)
ror['acronyms_len'] = ror['acronyms'].apply(len)
ror['labels_len'] = ror['labels'].apply(len)
ror['address_len'] = ror['addresses'].apply(len)
ror['address'] = ror['addresses'].apply(lambda x: x[0])
ror['ror_id'] = ror['id'].apply(lambda x: x.split("/")[-1])
ror['types'] = ror['types'].apply(lambda x: x[0])

In [194]:
ror[ror['ror_id']=='05kxf7578']

,id,name,types,links,aliases,acronyms,status,wikipedia_url,labels,email_address,ip_addresses,established,country,relationships,addresses,external_ids,alias_len,acronyms_len,labels_len,address_len,address,ror_id
23631,https://ror.org/05kxf7578,Sphinx (Netherlands),Company,[],[],[],active,,[],None,[],1834.0,"{'country_code': 'NL', 'country_name': 'Netherlands'}",[],"[{'line': None, 'lat': 50.856093, 'lng': 5.689349, 'postcode': None, 'primary': False, 'city': 'Maastricht', 'state': None, 'state_code': None, 'country_geonames_id': 2750405, 'geonames_city': {'id': 2751283, 'city': 'Maastricht', 'nuts_level1': {'code': 'NL4', 'name': 'ZUID-NEDERLAND'}, 'nuts_level2': {'code': 'NL42', 'name': 'Limburg (NL)'}, 'nuts_level3': {'code': 'NL423', 'name': 'Zuid-Limburg'}, 'geonames_admin1': {'id': 2751596, 'name': 'Limburg', 'ascii_name': 'Limburg', 'code': 'NL.05'}, 'geonames_admin2': {'id': 2751282, 'name': 'Gemeente Maastricht', 'ascii_name': 'Gemeente Maastricht', 'code': 'NL.05.0935'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}}}]","{'Wikidata': {'preferred': None, 'all': ['Q2738000']}, 'GRID': {'preferred': 'grid.435837.e', 'all': 'grid.435837.e'}}",0,0,0,1,"{'line': None, 'lat': 50.856093, 'lng': 5.689349, 'postcode': None, 'primary': False, 'city': 'Maastricht', 'state': None, 'state_code': None, 'country_geonames_id': 2750405, 'geonames_city': {'id': 2751283, 'city': 'Maastricht', 'nuts_level1': {'code': 'NL4', 'name': 'ZUID-NEDERLAND'}, 'nuts_level2': {'code': 'NL42', 'name': 'Limburg (NL)'}, 'nuts_level3': {'code': 'NL423', 'name': 'Zuid-Limburg'}, 'geonames_admin1': {'id': 2751596, 'name': 'Limburg', 'ascii_name': 'Limburg', 'code': 'NL.05'}, 'geonames_admin2': {'id': 2751282, 'name': 'Gemeente Maastricht', 'ascii_name': 'Gemeente Maastricht', 'code': 'NL.05.0935'}, 'license': {'attribution': 'Data from geonames.org under a CC-BY 3.0 license', 'license': 'http://creativecommons.org/licenses/by/3.0/'}}}",05kxf7578


In [10]:
# this file is not provided but the needed data is all institutions in OpenAlex
# with the following columns: 'ror_id','affiliation_id'
insts = pd.read_parquet("OA_static_institutions_single_file.parquet", 
                        columns=['affiliation_id','ror_id'])

In [11]:
insts.sample()

,affiliation_id,ror_id
69861,4210121867,02gbye032


In [10]:
ror_to_join = ror[['ror_id','name','aliases','acronyms','labels','country','types',
                   'address','alias_len','acronyms_len','labels_len','address_len']].copy()

In [11]:
def get_geoname_admin(address_dict):
    try:
        geoname_admin = address_dict['geonames_city']['geonames_admin1']['name']
    except:
        geoname_admin = "None"
        
    return geoname_admin

In [12]:
ror_to_join['country_code'] = ror_to_join['country'].apply(lambda x: x['country_code'])
ror_to_join['country_name'] = ror_to_join['country'].apply(lambda x: x['country_name'])
ror_to_join['city'] = ror_to_join['address'].apply(lambda x: x['city'])
ror_to_join['state'] = ror_to_join['address'].apply(lambda x: x['state'])
ror_to_join['region'] = ror_to_join['address'].apply(get_geoname_admin)
ror_to_join['institution'] = ror_to_join['name']

##### Looking at ROR Labels

In [14]:
codes_to_ignore = ['ja','fa','hi','ko','bn','zh','ml','ru','el','kn','gu','mk','ne','te','hy',
                   'km','ti','kk','th','my','uk','pa','bg','ur','vi','ar','sr','he','ta','ka',
                   'am','mr','lo','mn','be','or','ba','si','ky','uz']

In [15]:
labels = ror_to_join['labels'].explode().dropna().reset_index()
labels['label'] = labels['labels'].apply(lambda x: x['label'])
labels['iso639'] = labels['labels'].apply(lambda x: x['iso639'])

In [16]:
labels[~labels['iso639'].isin(codes_to_ignore)].sample(20)

,index,labels,label,iso639
21975,90310,"{'label': 'Region Hovedstadens Apotek', 'iso639': 'da'}",Region Hovedstadens Apotek,da
10450,39391,"{'label': 'Fachhochschule Stralsund', 'iso639': 'de'}",Fachhochschule Stralsund,de
8653,32284,"{'label': 'Biznesa augstskola Turība', 'iso639': 'lv'}",Biznesa augstskola Turība,lv
26569,99193,"{'label': 'Associação Para a Investigação Biomédica e Inovação Em Luz e Imagem', 'iso639': 'pt'}",Associação Para a Investigação Biomédica e Inovação Em Luz e Imagem,pt
26964,100515,"{'label': 'Sveučilište u Dubrovniku', 'iso639': 'hr'}",Sveučilište u Dubrovniku,hr
25276,96799,"{'label': 'Université d'économie d'hô-chi-minh-ville', 'iso639': 'fr'}",Université d'économie d'hô-chi-minh-ville,fr
4037,9230,"{'label': 'Centro Internacional de Restauración Neurológica', 'iso639': 'es'}",Centro Internacional de Restauración Neurológica,es
1146,1533,"{'label': 'Université d'État de chicago', 'iso639': 'fr'}",Université d'État de chicago,fr
5621,25645,"{'label': 'Hogeschool Utrecht', 'iso639': 'nl'}",Hogeschool Utrecht,nl
2476,4186,"{'label': 'Universitat CEU Cardenal Herrera', 'iso639': 'ca'}",Universitat CEU Cardenal Herrera,ca


#### Getting string beginnings

Looking to introduce more variety into the artificial strings so that they include some header information such as "College of .." or "Department of...".

In [17]:
with open("./ror_string_beginnings/Company", 'r') as f:
    company_begs = f.readlines()

company_begs = list(set([x.rstrip('\n') for x in company_begs]))

In [18]:
with open("./ror_string_beginnings/Education_dept", 'r') as f:
    education_dept_begs = f.readlines()

education_dept_begs = list(set([x.rstrip('\n') for x in education_dept_begs]))

In [19]:
with open("./ror_string_beginnings/Education_college", 'r') as f:
    education_col_begs = f.readlines()

education_col_begs = list(set([x.rstrip('\n') for x in education_col_begs]))

In [20]:
with open("./ror_string_beginnings/Education_school", 'r') as f:
    education_school_begs = f.readlines()

education_school_begs = list(set([x.rstrip('\n') for x in education_school_begs]))

In [21]:
with open("./ror_string_beginnings/Healthcare", 'r') as f:
    healthcare_begs = f.readlines()

healthcare_begs = list(set([x.rstrip('\n') for x in healthcare_begs]))

In [22]:
all_education = []
for beg in education_col_begs:
    all_education.append(f"College of {beg}")
    
for beg in education_dept_begs:
    all_education.append(f"Department of {beg}")
    all_education.append(f"Dep of {beg}")
    all_education.append(f"Dept of {beg}")
    all_education.append(f"Dep. of {beg}")
    all_education.append(f"Dept. of {beg}")
    
for beg in education_school_begs:
    all_education.append(f"School of {beg}")
    all_education.append(f"Sch. of {beg}")
    all_education.append(f"Sch of {beg}")
len(all_education)

272

In [23]:
all_company = company_begs

In [24]:
all_healthcare = []
for beg in healthcare_begs:
    all_healthcare.append(f"Department of {beg}")
    all_healthcare.append(f"Dep of {beg}")
    all_healthcare.append(f"Dept of {beg}")
    all_healthcare.append(f"Dep. of {beg}")
    all_healthcare.append(f"Dept. of {beg}")

##### Creating the artificial affiliation strings

We would like to take advantage of the ROR data and use it to supplement/augment the current affiliation string data in OpenAlex. This could potentially allow for the Institution Tagger model to predict on institutions that have not yet had affiliation strings added to OpenAlex.

In [25]:
type_preinst_dict = {'Company': all_company,
                     'Education': all_education,
                     'Healthcare': all_healthcare}

In [26]:
def create_affiliation_string_from_scratch(institution, city, state, country, region, aliases, labels, 
                                           acronyms, inst_type):
    aff_strings = []
    if aliases:
        aliases = [institution] + aliases
    else:
        aliases = [institution]
    if labels:
        for label in labels:
            if label['iso639'] in codes_to_ignore:
                pass
            else:
                aliases.append(label['label'])
    if acronyms:
        for acronym in acronyms:
            aliases.append(acronym)
    for alias in aliases:
        if "r&d" not in alias.lower():
            alias = alias.replace(" & ", " and ")
        match_string = unidecode.unidecode(alias)
        match_string = match_string.lower().replace(" ", "")
        match_string = "".join(re.findall("[a-zA-Z0-9]+", match_string)) 
        alias = unidecode.unidecode(alias)
        if ((state is None) & (region != city) & (city is not None) & 
            (country is not None) & (region is not None)):
            region = unidecode.unidecode(region)
            country = unidecode.unidecode(country)
            city = unidecode.unidecode(city)
            aff_strings.append([f"{alias}, {city}, {region}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {region}", match_string])
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}", match_string])
            aff_strings.append([f"{alias}, {region}", match_string])
            aff_strings.append([f"{alias} {city} {region} {country}", match_string])
            aff_strings.append([f"{alias} {city} {country}", match_string])
            aff_strings.append([f"{alias} {city} {region}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias} {city}", match_string])
            aff_strings.append([f"{alias} {region}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(5):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(2):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {region}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {region}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                
        elif (state is not None) & (city is not None) & (country is not None):
            state = unidecode.unidecode(state)
            country = unidecode.unidecode(country)
            city = unidecode.unidecode(city)
            aff_strings.append([f"{alias}, {city}, {state}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}, {state}", match_string])
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}", match_string])
            aff_strings.append([f"{alias}, {state}", match_string])
            aff_strings.append([f"{alias} {city} {state} {country}", match_string])
            aff_strings.append([f"{alias} {city} {country}", match_string])
            aff_strings.append([f"{alias} {city} {state}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias} {city}", match_string])
            aff_strings.append([f"{alias} {state}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(1):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {state}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {state}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
        elif (city is not None) & (country is not None):
            country = unidecode.unidecode(country)
            city = unidecode.unidecode(city)
            aff_strings.append([f"{alias}, {city}, {country}", match_string])
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias}, {city}", match_string])
            aff_strings.append([f"{alias} {city} {country}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias} {city}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(1):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {city}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {city}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
        elif (country is not None):
            country = unidecode.unidecode(country)
            aff_strings.append([f"{alias}, {country}", match_string])
            aff_strings.append([f"{alias} {country}", match_string])
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(4):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                else:
                    for i in range(1):
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}, {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]} {country}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}, {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias} {country}", match_string])
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                
        else:
            aff_strings.append([f"{alias}", match_string])
            if inst_type in ['Education','Company','Healthcare']:
                list_to_sample = type_preinst_dict[inst_type]
                if inst_type == 'Education':
                    for i in range(5):
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                elif inst_type == 'Healthcare':
                    for i in range(3):
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
                else:
                    for i in range(2):
                        aff_strings.append([f"{sample(list_to_sample, 1)[0]} {alias}", match_string])
                        aff_strings.append([f"{alias} {sample(list_to_sample, 1)[0]}", match_string])
    return aff_strings

In [27]:
def get_institutions_list(institution, aliases, labels, acronyms):
    aff_strings = []
    if aliases:
        aliases = [institution] + aliases
    else:
        aliases = [institution]
    if labels:
        for label in labels:
            if label['iso639'] in codes_to_ignore:
                pass
            else:
                aliases.append(label['label'])
    return aliases

In [28]:
ror_to_join['affs'] = ror_to_join \
.apply(lambda x: get_institutions_list(x.institution, x.aliases,
                                       x.labels, x.acronyms), axis=1)
ror_to_join['aff_string'] = ror_to_join \
.apply(lambda x: create_affiliation_string_from_scratch(x.institution, x.city, x.state, 
                                                        x.country_name, x.region, 
                                                        x.aliases, x.labels, x.acronyms, x.types), axis=1)
ror_to_join['aff_string_len'] = ror_to_join['aff_string'].apply(len)
ror_to_join_final = ror_to_join.explode("aff_string").copy()

##### Looking to quickly get combinations of city/region/country

In [29]:
art_empty_affs = ror_to_join[['city','region','country_name']].dropna().copy()

In [30]:
art_empty_affs.shape

(104323, 3)

In [31]:
art_empty_affs.sample(10)

,city,region,country_name
40310,Austin,Texas,United States
101513,Agra,Uttar Pradesh,India
76749,Seoul,Seoul,South Korea
66128,Beijing,Beijing,China
3685,Chiayi City,Taiwan,Taiwan
25248,Dresden,Saxony,Germany
76669,Sejong,Sejong-si,South Korea
38275,Beijing,Beijing,China
44759,Al Rayyan,Baladīyat ar Rayyān,Qatar
8592,Springfield,Missouri,United States


In [32]:
art_empty_affs['original_affiliation_1'] = \
    art_empty_affs.apply(lambda x: f"{x.city}, {x.country_name}", axis=1)

In [33]:
art_empty_affs['original_affiliation_2'] = \
    art_empty_affs.apply(lambda x: f"{x.city}, {x.region}, {x.country_name}", axis=1)

In [34]:
city_country = art_empty_affs.sample(1500).drop_duplicates()\
    ['original_affiliation_1'].to_list()

In [36]:
city_region_country = art_empty_affs.sample(1500).drop_duplicates()\
    ['original_affiliation_2'].to_list()

Some of these string will be used to train the model that only seeing a city/region/country should be a "no prediction"

In [38]:
pd.DataFrame(zip(city_country+city_region_country), 
             columns=['original_affiliation']) \
    .drop_duplicates(subset=['original_affiliation'])\
    .to_parquet("artificial_empty_affs.parquet")

In [39]:
ror_to_join_final = ror_to_join[['ror_id','aff_string']].explode("aff_string").copy()
ror_to_join_final.shape

(9798246, 2)

In [40]:
ror_to_join_final['original_affiliation'] = \
    ror_to_join_final['aff_string'].apply(lambda x: x[0])

The rest of the artificial strings are exported so that they can be combined with the historical affiliation data to create the final training data set.

In [46]:
ror_to_join_final.merge(insts, how='inner', 
                        on='ror_id')[['original_affiliation','affiliation_id']] \
.to_parquet("s3://institutional-affiliation-classification/V3/mag_and_ror_aff_strings_data/ror_strings.parquet")